The purpose of this notbook is to augments dataset, to enhance model context 

In [1]:
import os
import json
import time
from pathlib import Path
from dotenv import load_dotenv
from groq import Groq

def GenerationGroq(prompt, groq_key, temperature=0.7, max_tokens=8192):
    client = Groq(api_key=groq_key)

    completion = client.chat.completions.create(
        model="openai/gpt-oss-safeguard-20b",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature,
        max_completion_tokens=max_tokens,
        top_p=1,
        reasoning_effort="medium",
        stream=True
    )

    generated = ""
    for chunk in completion:
        piece = chunk.choices[0].delta.content or ""
        print(piece, end="")
        generated += piece

    return generated.strip()

In [2]:
load_dotenv()
GROQ_KEY = os.getenv("groq_api")

if not GROQ_KEY:
    raise ValueError("Missing 'groq_api' in your .env file")

base_folder = Path("data_final")

WAIT_TIME = 2
MAX_RETRIES = 3

print(f"Augmenting files in folder: {base_folder}\n")

for folder in os.listdir(base_folder):
    folderP = base_folder / folder

    if not folderP.is_dir():
        continue

    for file in os.listdir(folderP):
        if file.endswith("_reph.txt"):
            try:
                (folderP / file).unlink()
                print(f"Deleted old file: {file}")
            except Exception as e:
                print(f"Could not delete {file}: {e}")

    for file in os.listdir(folderP):
        file_path = folderP / file

        if not file_path.is_file():
            continue

        print(f"\nProcessing: {file_path.name}\n")

        try:
            # Load content
            if file_path.suffix == ".json":
                with open(file_path, "r", encoding="utf-8") as f:
                    original_data = json.load(f)
                    content = json.dumps(original_data, ensure_ascii=False)
            else:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()

        except Exception as e:
            print(f"Could not read {file_path.name}, error: {e}")
            continue

        # Prompt
        prompt = (
            "Reformule le texte suivant sans en modifier le sens. "
            "Utilise des mots, des tournures et des structures de phrases différentes. "
            "N’ajoute AUCUN symbole markdown, ni gras, ni italique, ni titres, ni listes, "
            "ni blocs de code, ni backticks, ni caractères décoratifs comme **** ou ----. "
            "Pour les emplois du temps (fichiers json), transforme-les en un paragraphe descriptif. "
            "N’ajoute aucune explication, aucun commentaire, ni contenu supplémentaire. "
            "Renvoie UNIQUEMENT le texte reformulé, rien d’autre."
            "La reformulation doit être en français.\n\n"
            f"{content}"
        )



        # Retry logic
        for attempt in range(1, MAX_RETRIES + 1):
            try:
                augmented = GenerationGroq(prompt, GROQ_KEY)
                break
            except Exception as err:
                print(f"Attempt {attempt} failed: {err}")
                time.sleep(WAIT_TIME * attempt)
        else:
            print(f"Failed to augment {file_path.name}")
            continue

        # Save result as .txt
        new_name = file_path.stem + "_reph.txt"
        new_path = folderP / new_name

        try:
            with open(new_path, "w", encoding="utf-8") as f:
                f.write(augmented)

            print(f"Saved augmented file: {new_path.name}")

        except Exception as e:
            print(f"Error saving augmented file for {file_path.name}, error: {e}")
            continue


Augmenting files in folder: data_final


Processing: calendrier-printemps-2024-2025.txt

Les événements du semestre de printemps 2024/2025 comprennent le contrôle continu 1 le 15 février, l'arrêt de l'enseignement le 17 mars, le début des contrôles finaux le 24 avril, la Fête du Travail le 1er mai, les délibérations du CI3 le 2 juin, celles du CI2 le 3 juin, celles du CI1 le 4 juin, les délibérations de 2AP1 le 7 juin et celles de 2AP2 le 8 juin, le premier Moharram le 27 juin, le début des soutenances des PFEs le 28 juin, la Fête du Trône le 30 juillet, ainsi que les congés religieux et les vacances dont les dates précises restent à confirmer, notamment Aid El Fitr, le congé de printemps, les contrôles de rattrapage, Aid Al Adha, et les vacances de fin du premier semestre.Saved augmented file: calendrier-printemps-2024-2025_reph.txt

Processing: les clubs de l'ecole.txt

Les clubs de l'ENSA Tétouan se présentent comme suit :  
Presse ENSA Tétouan regroupe des étudiants qui souhaitent 